In [357]:
import warnings
warnings.filterwarnings("ignore")

In [358]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from pymystem3 import Mystem
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from itertools import groupby

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\potkd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\potkd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\potkd\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\potkd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [359]:
pd.options.display.max_colwidth = 100

In [360]:
data = pd.read_excel('data.xlsx')

In [361]:
data

,plan_name,kod,text
0,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-1,"Способен осуществлять поиск, критический анализ и синтез информации, применять системный подход ..."
1,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-1.1,"Осуществляет поиск необходимой информации, опираясь на результаты анализа поставленной задачи"
2,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-1.2,Разрабатывает варианты решения проблемной ситуации на основе критического анализа доступных исто...
3,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-1.3,"Выбирает оптимальный вариант решения задачи, аргументируя свой выбор"
4,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-2,Способен определять круг задач в рамках поставленной цели и выбирать оптимальные способы их реше...
...,...,...,...
4130,45.03.02_2023_Л_ПиПвСЭиФФЯ_О_01,ПК-3.1,Применяет адекватные стратегии перевода и техники компрессии при осуществлении устного профессио...
4131,45.03.02_2023_Л_ПиПвСЭиФФЯ_О_01,ПК-3.2,Умеет использовать вербальные и невербальные средства языка в зависимости от\nкультурологическог...
4132,45.03.02_2023_Л_ПиПвСЭиФФЯ_О_01,ПК-4,Способен осуществлять организацию управления качеством перевода
4133,45.03.02_2023_Л_ПиПвСЭиФФЯ_О_01,ПК-4.1,"Умеет проводить экспертизу устного и письменного перевода, в том числе документации, и давать эк..."


### Сервис для препроцессинга

In [362]:
class PreprocessingService:
    
    CLASS_1 = [
    'Применяет прикладное программное обеспечение для решения профессиональных задач',
    'Применяет современные инструментальные средства и языки программирования для решения задач интеллектуального анализа данных',
    'Знает основные стандарты оформления технической документации на различных стадиях жизненного цикла информационной системы',
    'Имеет навыки внедрения искусственного интеллекта в управление цифровыми платформами',
    'Умеет разрабатывать мобильные приложения',
    'Способен разрабатывать алгоритмы и компьютерные программы, пригодные для практического применения',
    'Способен проектировать программное обеспечение для информационно-коммуникационных систем'
    ]
    
    CLASS_0 = [
    'Способен осуществлять социальное взаимодействие и реализовывать свою роль в команде',
    'Понимает основные аспекты межличностных и групповых коммуникаций',
    'Применяет методы командного взаимодействия',
    'Способен осуществлять деловую коммуникацию в устной и письменной формах на государственном языке Российской Федерации и иностранном(ых) языке(ах)',
    'Способен воспринимать межкультурное разнообразие общества в социально-историческом, этическом и философском контекстах',
    'Проявляет в своем поведении уважительное отношение к историческому наследию и социокультурным традициям различных социальных групп, опирающееся на знание этапов исторического развития России в контексте мировой истории и культурных традиций мира',
    'Способен принимать обоснованные экономические решения в различных областях жизнедеятельности'
    ]
    
    UNION_STOPWORDS_SET = {'знать', 'иметь', 'проявлять', 'способный', 'принимать', 'применять'}
    
    def __init__(self, df, column_text='text', column_class='class', status='inference', vectorizer=None):
        _df = df.copy()
        self.df = _df
        self.column_text = column_text
        self.column_class = column_class
        self.status = status
        self.vectorizer = vectorizer
        self.class_1 = self.CLASS_1
        self.class_0 = self.CLASS_0
        self.union_stopwords_set = self.UNION_STOPWORDS_SET
        
    def _classifier(self, row):
        
        if row[self.column_text] in self.class_1:
            val = 1
        
        elif row[self.column_text] in self.class_0:
            val = 0

        else:
            val = np.nan
    
        return val
    
    def _get_stopwords(self):
        
        stopwords = set(nltk.corpus.stopwords.words('russian'))
        stopwords = stopwords.union(self.union_stopwords_set)
        
        return stopwords
        
    
    def _lemmatization_massive(self, texts):
        
        alltexts  = '' 
        for txt in texts:
            alltexts = alltexts + ' brake ' + txt
        alltexts = Mystem().lemmatize(alltexts)
        alltexts = list(filter((' ').__ne__, alltexts))
        alltexts = list(filter(('\n').__ne__, alltexts))
        
        return [' '.join(list(g)) for k, g in groupby(alltexts, key=lambda x: x != "brake") if k]
    
    def _get_matrix(self, series, vectorizer):
        
        text = series[self.column_text].tolist()
        
        if self.status == 'train': 
            matrix = vectorizer.fit_transform(text).toarray()
        else:
            matrix = vectorizer.transform(text).toarray()
        
        return matrix
        
    
    def run(self):
        if self.status == 'train':
            self.df[self.column_class] = self.df.apply(self._classifier, axis=1)
            self.df = self.df[~self.df[self.column_class].isna()]
            # self.df = self.df.groupby(self.column_text).mean().reset_index()[[self.column_text, self.column_class]]
            # при группировке (по сути удалению дубликатов) по столбцу с текстом проседает метрика при обучении 
            # цифры при инференсе на всём датасете становятся, мой взгляд, менее реалистичными, так как сдаётся мне 
            # цифровых компетенций существенно меньше всех прочих
            # если не группировать получаем высокое качество на обучении и опять же, на мой взгляд, более реальные цифры
            # для всего датасета, при том что по сути модели никаких новых уникальных данных не получают
            # в любом случае было бы хорошо в ручную разметить большую часть датасета
        self.df[self.column_text] = self.df[self.column_text].apply(lambda x: re.sub("[^А-Яа-яЁё ]+", " ", x).lower())
        self.df[self.column_text] = self._lemmatization_massive(self.df[self.column_text].tolist())
        self.df[self.column_text] = self.df[self.column_text].apply(lambda x: nltk.word_tokenize(x, language = "russian"))
        stopwords = self._get_stopwords()
        self.df[self.column_text] = self.df[self.column_text].apply(lambda x: [t for t in x if t not in stopwords])
        self.df[self.column_text] = self.df[self.column_text].apply(lambda x: ' '.join(x))
        if self.status == 'train':
            vectorizer = CountVectorizer(stop_words=list(stopwords))
        else:
            vectorizer = self.vectorizer
        matrix = self._get_matrix(self.df, vectorizer)
        if self.status == 'train':
            X = matrix
            y = self.df[self.column_class]
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
            return vectorizer, X_train, X_test, y_train, y_test
        else:
            X = matrix
            return X

### Обучение моделей

In [363]:
preprocessing = PreprocessingService(df=data, status='train')
vectorizer, X_train, X_test, y_train, y_test = preprocessing.run()

In [364]:
data

,plan_name,kod,text
0,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-1,"Способен осуществлять поиск, критический анализ и синтез информации, применять системный подход ..."
1,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-1.1,"Осуществляет поиск необходимой информации, опираясь на результаты анализа поставленной задачи"
2,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-1.2,Разрабатывает варианты решения проблемной ситуации на основе критического анализа доступных исто...
3,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-1.3,"Выбирает оптимальный вариант решения задачи, аргументируя свой выбор"
4,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-2,Способен определять круг задач в рамках поставленной цели и выбирать оптимальные способы их реше...
...,...,...,...
4130,45.03.02_2023_Л_ПиПвСЭиФФЯ_О_01,ПК-3.1,Применяет адекватные стратегии перевода и техники компрессии при осуществлении устного профессио...
4131,45.03.02_2023_Л_ПиПвСЭиФФЯ_О_01,ПК-3.2,Умеет использовать вербальные и невербальные средства языка в зависимости от\nкультурологическог...
4132,45.03.02_2023_Л_ПиПвСЭиФФЯ_О_01,ПК-4,Способен осуществлять организацию управления качеством перевода
4133,45.03.02_2023_Л_ПиПвСЭиФФЯ_О_01,ПК-4.1,"Умеет проводить экспертизу устного и письменного перевода, в том числе документации, и давать эк..."


#### Наивный Байес

In [365]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train, y_train)
nb.score(X_test, y_test)

0.9904761904761905

#### Логистическая регрессия

In [366]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
logreg.score(X_test, y_test)

0.9523809523809523

#### Метод опорных векторов

In [367]:
from sklearn import svm

metodsvm = svm.SVC()
metodsvm.fit(X_train, y_train)
metodsvm.score(X_test, y_test)

0.9523809523809523

#### Градиентный бустинг

In [368]:
from sklearn.ensemble import GradientBoostingClassifier

boost = GradientBoostingClassifier()
boost.fit(X_train, y_train)
boost.score(X_test, y_test)

0.9523809523809523

### Инференс моделей

In [369]:
preprocessing = PreprocessingService(df=data, vectorizer=vectorizer)
X = preprocessing.run()

#### Наивный Байес

In [370]:
pd.Series(nb.predict(X)).value_counts()

0.0    2420
1.0    1715
dtype: int64

#### Логистическая регрессия

In [371]:
pd.Series(logreg.predict(X)).value_counts()

0.0    4133
1.0       2
dtype: int64

#### Метод опорных векторов

In [372]:
pd.Series(metodsvm.predict(X)).value_counts()

0.0    4133
1.0       2
dtype: int64

#### Градиентный бустинг

In [373]:
pd.Series(boost.predict(X)).value_counts()

0.0    4033
1.0     102
dtype: int64

**_Наиболее реальным по моим субъективным представлениям о соотношении цифровых и прочих компетенций является прогноз модели градиентного бустинга._**

In [374]:
data

,plan_name,kod,text
0,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-1,"Способен осуществлять поиск, критический анализ и синтез информации, применять системный подход ..."
1,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-1.1,"Осуществляет поиск необходимой информации, опираясь на результаты анализа поставленной задачи"
2,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-1.2,Разрабатывает варианты решения проблемной ситуации на основе критического анализа доступных исто...
3,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-1.3,"Выбирает оптимальный вариант решения задачи, аргументируя свой выбор"
4,01.03.02_2023_ПМ_ПМиИвЭиУ_ОФО,УК-2,Способен определять круг задач в рамках поставленной цели и выбирать оптимальные способы их реше...
...,...,...,...
4130,45.03.02_2023_Л_ПиПвСЭиФФЯ_О_01,ПК-3.1,Применяет адекватные стратегии перевода и техники компрессии при осуществлении устного профессио...
4131,45.03.02_2023_Л_ПиПвСЭиФФЯ_О_01,ПК-3.2,Умеет использовать вербальные и невербальные средства языка в зависимости от\nкультурологическог...
4132,45.03.02_2023_Л_ПиПвСЭиФФЯ_О_01,ПК-4,Способен осуществлять организацию управления качеством перевода
4133,45.03.02_2023_Л_ПиПвСЭиФФЯ_О_01,ПК-4.1,"Умеет проводить экспертизу устного и письменного перевода, в том числе документации, и давать эк..."


In [375]:
data_predict = data.copy()
data_predict['class'] = boost.predict(X)

In [376]:
data_predict.to_excel('data_predict.xlsx')